In [1]:
from unsloth import FastLanguageModel
from transformers import AutoModelForSequenceClassification

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = 'mistral_lora_model',
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
        local_files_only = True,
    )

In [ ]:
# Push the model to your Hugging Face repository
token = '' #HF token
model.push_to_hub("",tokenizer, save_method = "lora", token = token) 

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
def extract_response(text):
    # Split the input by the "Response:" keyword
    response_key = "### Response:"
    if response_key in text:
        response = text.split(response_key, 1)[-1].strip()
        return response[:-10]
    else:
        return "No response found."

def generate_text(model,tokenizer, text, max_length=256):
    """Function to generate text using the model"""
    FastLanguageModel.for_inference(model)
    # Generate predictions
    instruction = """Perform Diagnosis"""
    
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction, # instruction
            text, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = max_length, use_cache = True)
    text= tokenizer.batch_decode(outputs)
    return extract_response(text[0])

In [13]:
text = ''' Age:40 , Sex:Male, Symptoms: Diarrhea.
Fever.
Fatigue.
Belly pain and cramping.
Blood in the stool.
Mouth sores.
Reduced appetite and weight loss.
Pain or drainage near or around the anus due to inflammation from a tunnel into the skin, called a fistula.'''
x = generate_text(model,tokenizer, text, 500)
print(x)

Differential Diagnosis is:
Colitis, HIV (initial infection), Anemia, Pancreatic neoplasm and the most likely is Colitis
